In [ ]:
import os
import numpy as np
import pandas as pd
import json
import itertools
import pickle
import matplotlib.pyplot as plt
import torch
from sentence_transformers import SentenceTransformer, util

## Prepare data

### Selecte data relevant to federal races

In [ ]:
source_path = '../../../../fb_2022'

In [ ]:
var = pd.read_csv(os.path.join(source_path, 'fb_2022_adid_var_sets_091224.csv.gz'))

In [ ]:
var.columns

In [ ]:
'''
Filter for Set 3
'''
var = var[var.set3 == 1]

In [ ]:
var[~var.race_of_focus.isin(['No race of focus', 'Downballot'])].sub_bucket.unique()

In [ ]:
'''
Only house/senate races 
'''
condition = (var.wmp_office.isin(['us senate', 'us house'])) | (~var.race_of_focus.isin(['No race of focus', 'Downballot']))

In [ ]:
var = var[condition]

In [ ]:
var.shape

In [ ]:
var.groupby(['wmp_spontype']).size()

In [ ]:
'''
Merge with text and cid mapping tables
'''
cid = pd.read_csv(os.path.join(source_path, 'cid_fb2022.csv'))
text = pd.read_csv(os.path.join(source_path, 'fb_2022_adid_text.csv.gz'))

In [ ]:
var = var.merge(text, how='left', on='ad_id')

In [ ]:
var = var.merge(cid, how='left', on='ad_id')

In [ ]:
var[pd.isna(var.federal_cd)].shape

In [ ]:
'''
Drop clearly non-federal race spnosors
'''
var = var[var.wmp_spontype!= 'down ballot']

### Aggregate data at sponsor-media level

#### Federal candidates

In [ ]:
var_cand = var[var.federal_cd==1]

In [ ]:
cols = ['pd_id', 'wmp_media_type', 'cid']

In [ ]:
# Aggregate by pdid AND media type
cand_sponsor_agg = var_cand[cols].groupby(cols).first().reset_index().groupby(['pd_id', 'wmp_media_type']).agg(lambda x: ' '.join(x)).reset_index()

In [ ]:
cand_sponsor_agg.head(2)

In [ ]:
cand_sponsor_agg['num_unique'] = cand_sponsor_agg.cid.apply(lambda x: len(x.split()))

In [ ]:
cand_sponsor_agg = cand_sponsor_agg[cand_sponsor_agg.num_unique > 1]

In [ ]:
var_cand.wmp_spontype.unique()

#### Non-campaign sponsors: Advertiser-media-race_of_focuse-level aggregation

In [ ]:
var[var.federal_cd == 0].wmp_spontype.unique()

In [ ]:
var[var.federal_cd == 0].wmp_office.unique()

In [ ]:
'''
filter irrelevant offices among non-campaign sponsors
'''
irrelevant_offices = ['us house - other cycle', 'us senate - other cycle', 'down ballot', 
 'president - other cycle', 'ballot measure', 'election outside US or in US comm/territory', 'governor']
condition3 = (var.federal_cd == 0) & (~var.wmp_office.isin(irrelevant_offices))

var_noncand = var[condition3]

In [ ]:
'''
Remove those with no race of focus identified
'''
var_noncand = var_noncand[~pd.isna(var_noncand.race_of_focus)]
var_noncand = var_noncand[var_noncand.race_of_focus != 'No race of focus']

In [ ]:
'''
Aggregate at sponsor-media-race level
'''
cols2 = ['pd_id', 'cid', 'race_of_focus', 'wmp_media_type']

In [ ]:
'''
Agg by pdid AND media type AND race of focus
'''
noncand_sponsor_agg = var_noncand[cols2].groupby(['pd_id', 'cid', 'race_of_focus']).first().reset_index()\
.groupby(['pd_id', 'race_of_focus', 'wmp_media_type']).agg(lambda x: ' '.join(x)).reset_index()

In [ ]:
noncand_sponsor_agg.head(2)

In [ ]:
noncand_sponsor_agg['num_unique'] = noncand_sponsor_agg.cid.apply(lambda x: len(x.split()))

# filter for sponsors with at least a pair of unique creatives
noncand_sponsor_agg = noncand_sponsor_agg[noncand_sponsor_agg.num_unique > 1]

In [ ]:
noncand_sponsor_agg.shape

### Import trained corpous embedding and compute average similarity

In [ ]:
corpus_embeddings = torch.load('../input_data/corpus_embedding_fb2022_uni.pt', map_location=torch.device('cpu'))

In [ ]:
corpus_embeddings.shape
# corpus embedding index aligned with ascending cid index starting from 0 

In [ ]:
def compute_avg_similarity(cid_lst):
    cid_lst = cid_lst.split()
    
    pairs = list(itertools.combinations(cid_lst, 2))
    if pairs:
        scores = []
        for pair in pairs: 
            cid1, cid2 = pair
            idx1 = int(cid1.lstrip('cid_'))
            idx2 = int(cid2.lstrip('cid_'))
            
            score = util.cos_sim(corpus_embeddings[idx1], corpus_embeddings[idx2])[0][0].numpy()
            scores.append(score)
        return np.array(scores).mean(), np.array(scores).std()
    return None

### Federal candidates: by advertiser and media type¶

In [ ]:
cand_sponsor_agg.loc[:, ['avg', 'std']] = cand_sponsor_agg.cid.apply(lambda x: pd.Series(compute_avg_similarity(x), index=['avg', 'std'], dtype="float"))

In [ ]:
cand_sponsor_agg.head(4)

In [ ]:
'''
Candidates only
'''

cand_sponsor_agg.to_csv('../output_data/fb_set3_cand_pdid_media_average_pairwise_similarity.csv', index=False)

### non-campaign candidate sponsors: by sponsor - media - race of focus


In [ ]:
noncand_sponsor_agg.loc[:, ['avg', 'std']] = noncand_sponsor_agg.cid.apply(lambda x: pd.Series(compute_avg_similarity(x), index=['avg', 'std'], dtype="float"))

In [ ]:
'''
Got the final avg text similarity of noncandidates grouped by race of focus average pairwise similarity
'''
noncand_sponsor_agg.to_csv('../output_data/fb_set3_noncandidate_pdid_racefocus_media_average_pairwise_similarity.csv', index=False)